In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_stones"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Kidney_stones/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Kidney_stones/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_stones/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Kidney_stones/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Kidney stones
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Kidney stones are a renal condition, so we should look for kidney-related cohorts
kidney_related_terms = ['kidney', 'renal', 'nephro']

# First check for direct kidney related cohorts
kidney_related_dirs = [d for d in available_dirs if any(term in d.lower() for term in kidney_related_terms)]
print(f"Kidney-related cohorts: {kidney_related_dirs}")

if not kidney_related_dirs:
    print(f"No suitable cohort found for {trait}.")
    # Mark the task as completed by recording the unavailability
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    # Exit the script early since no suitable cohort was found
    selected_cohort = None
else:
    # We have multiple kidney-related datasets, choose the one with the most likely relevance to kidney stones
    # For kidney stones, the clear cell or papillary types might be more relevant
    # Prioritize KIRC (clear cell) as it's the most common form of kidney cancer and might have the largest dataset
    if any('clear' in d.lower() for d in kidney_related_dirs):
        selected_cohort = [d for d in kidney_related_dirs if 'clear' in d.lower()][0]
    else:
        selected_cohort = kidney_related_dirs[0]

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Kidney_stones...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'T


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_KIRC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'disease_code', 'eastern_cancer_oncology_group', 'erythrocyte_sedimentation_rate_result', 'followup_case_report_form_submissi

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify columns related to age
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]

# Identify columns related to gender
candidate_gender_cols = ["gender"]

# Let's preview these columns
import pandas as pd

# First, get the clinical data file path
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)"))

# Read the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age-related columns
if candidate_age_cols:
    age_preview = {}
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    print("Age columns preview:")
    print(age_preview)

# Preview gender-related columns
if candidate_gender_cols:
    gender_preview = {}
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    print("\nGender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [69, 68, 67, 67, 66], 'days_to_birth': [-25205.0, -25043.0, -24569.0, -24569.0, -24315.0]}

Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'MALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Check age columns
age_columns = {'age_at_initial_pathologic_diagnosis': [69, 68, 67, 67, 66], 
               'days_to_birth': [-25205.0, -25043.0, -24569.0, -24569.0, -24315.0]}

# Check gender columns
gender_columns = {'gender': ['MALE', 'FEMALE', 'MALE', 'MALE', 'MALE']}

# Select age column - both seem valid, but age_at_initial_pathologic_diagnosis is more directly usable
age_col = 'age_at_initial_pathologic_diagnosis' if age_columns else None

# Select gender column - only one option
gender_col = 'gender' if gender_columns and len(gender_columns['gender']) > 0 else None

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy().T

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data instead - samples as rows, genes as columns
        normalized_gene_df = genetic_df
    else:
        # If normalization worked, transpose back to original orientation
        normalized_gene_df = normalized_gene_df.T
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Use is_final=False mode which doesn't require df and is_biased
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True
    )
    print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in trait and demographic features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Kidney Clear Cell Carcinoma cohort used for Kidney_stones gene expression analysis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Kidney_stones  Age  Gender
sampleID                                   
TCGA-3Z-A93Z-01              1   69       1
TCGA-6D-AA2E-01              1   68       0
TCGA-A3-3306-01              1   67       1
TCGA-A3-3306-11              0   67       1
TCGA-A3-3307-01              1   66       1

Processing gene expression data...


Original gene data shape: (20530, 606)


Attempting to normalize gene symbols...
Gene data shape after normalization: (0, 20530)
Using original gene data instead of normalized data.


Gene data saved to: ../../output/preprocess/Kidney_stones/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (945, 3)
Genetic data shape: (20530, 606)
Number of common samples: 606

Linked data shape: (606, 20533)
Linked data preview (first 5 rows, first few columns):
                 Kidney_stones  Age  Gender  ARHGEF10L     HIF3A
TCGA-CZ-4853-01              1   82       1   1.290608  0.845674
TCGA-CZ-5986-11              0   61       1   0.791108  1.333974
TCGA-BP-4762-01              1   42       1   0.051008 -1.774526
TCGA-B0-4828-01              1   79       1   0.716308  1.727174
TCGA-A3-3370-01              1   48       0   0.878308  1.101174



Data shape after handling missing values: (606, 20533)

Checking for bias in features:
For the feature 'Kidney_stones', the least common label is '0' with 72 occurrences. This represents 11.88% of the dataset.
The distribution of the feature 'Kidney_stones' in this dataset is fine.

Quartiles for 'Age':
  25%: 52.0
  50% (Median): 61.0
  75%: 70.0
Min: 26
Max: 90
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 208 occurrences. This represents 34.32% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Kidney_stones/TCGA.csv
Clinical data saved to: ../../output/preprocess/Kidney_stones/clinical_data/TCGA.csv
